# Try model


In [ ]:
! pip install ultralytics

import ultralytics
ultralytics.checks()

Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 34.5/112.6 GB disk)


In [ ]:
from ultralytics import YOLO
model = YOLO('/content/last.pt')

In [ ]:
image_path = '/content/photo_2024-12-05_20-33-18.jpg'

results = model(image_path)


Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Fusing layers... 
gelan-c summary: 467 layers, 25412502 parameters, 0 gradients, 102.5 GFLOPs


TypeError: DetectionModel.forward() got an unexpected keyword argument 'embed'

# Train model

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="kACovgXsfqfPRnZZuqtL")
project = rf.workspace("vbit-qfkwu").project("helmet-detection-kd5te")
version = project.version(4)
dataset = version.download("yolov9")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Helmet-detection-4 in yolov9:: 100%|██████████| 32912/32912 [00:05<00:00, 5492.22it/s]


In [ ]:
%cd {dataset.location}

yaml_content = """
names:
- head
- helmet
nc: 2
roboflow:
  license: CC BY 4.0
  project: helmet-detection-kd5te
  url: https://universe.roboflow.com/vbit-qfkwu/helmet-detection-kd5te/dataset/4
  version: 4
  workspace: vbit-qfkwu
test: /content/Helmet-detection-4/test/images
train: /content/Helmet-detection-4/train/images
val: /content/Helmet-detection-4/valid/images
"""

with open('data.yaml', 'w') as file:
    file.write(yaml_content)

%cd /content/

/content/Helmet-detection-4
/content


In [ ]:
from ultralytics import YOLO
import torch

device: str = "cuda" if torch.cuda.is_available() else "cpu"

print(f"device = {device}")

# Load a model
model = YOLO("yolo11n.yaml")  # build a new model from scratch
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
model.to(device)

# Use the model
model.train(data=f"{dataset.location}/data.yaml", epochs=100, imgsz=640, save_period=10, plots=True)  # train the model
metrics = model.val()  # evaluate model performance on the validation set

device = cuda
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/Helmet-detection-4/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=cuda:0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=Fal

train: Scanning /content/Helmet-detection-4/train/labels... 15061 images, 760 backgrounds, 0 corrupt: 100%|██████████| 15061/15061 [00:07<00:00, 1952.00it/s]


train: New cache created: /content/Helmet-detection-4/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/Helmet-detection-4/valid/labels... 1341 images, 81 backgrounds, 0 corrupt: 100%|██████████| 1341/1341 [00:01<00:00, 883.21it/s] 


val: New cache created: /content/Helmet-detection-4/valid/labels.cache
Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     29/100      2.65G        1.4     0.9007      1.239         10        640: 100%|██████████| 942/942 [05:18<00:00,  2.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:13<00:00,  3.16it/s]

                   all       1341       3726       0.87      0.824      0.875      0.536



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     30/100      2.54G      1.408     0.8962      1.235         57        640:  35%|███▌      | 330/942 [01:54<03:31,  2.89it/s]


KeyboardInterrupt: 

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("/content/runs/detect/train2/weights/best.pt")  # load a partially trained model


In [ ]:
model.predict(source="/content/photo_2024-12-05_20-33-18.jpg", conf=0.2,  save=True)


image 1/1 /content/photo_2024-12-05_20-33-18.jpg: 640x640 1 head, 1 helmet, 13.8ms
Speed: 2.6ms preprocess, 13.8ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict5


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'head', 1: 'helmet'}
 obb: None
 orig_img: array([[[252, 255, 253],
         [252, 255, 253],
         [252, 255, 253],
         ...,
         [252, 255, 253],
         [252, 255, 253],
         [252, 255, 253]],
 
        [[252, 255, 253],
         [252, 255, 253],
         [252, 255, 253],
         ...,
         [252, 255, 253],
         [252, 255, 253],
         [252, 255, 253]],
 
        [[252, 255, 253],
         [252, 255, 253],
         [252, 255, 253],
         ...,
         [252, 255, 253],
         [252, 255, 253],
         [252, 255, 253]],
 
        ...,
 
        [[251, 255, 255],
         [251, 255, 255],
         [251, 255, 255],
         ...,
         [252, 255, 253],
         [252, 255, 253],
         [252, 255, 253]],
 
        [[251, 255, 255],
         [251, 255, 255],
         [251, 255, 255],
         ...,
       

# Blurring proccess

In [1]:
! pip install -q ultralytics
! pip install -q opencv-python

import ultralytics
ultralytics.checks()

Ultralytics 8.3.47 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.6/112.6 GB disk)


In [2]:
from ultralytics import YOLO

model = YOLO('/content/best.pt')
example_photo_path = '/content/photo_2024-12-05_20-33-18.jpg'
example_photo_path_2 = '/content/photo_2024-12-06_21-05-01.jpg'


FileNotFoundError: [Errno 2] No such file or directory: '/content/best.pt'

In [ ]:
import cv2
import numpy as np

image = cv2.imread(example_photo_path)
image_2 = cv2.imread(example_photo_path_2)

results = model.predict(image, conf=0.2)
results_2 = model.predict(image_2, conf=0.2)
results[0].boxes

In [ ]:
import cv2

def blur_zones(image, zones):
    """
    Blurs specific zones in an image.

    Args:
        image (numpy.ndarray): The input image (cv2 image in BGR format).
        zones (list): A list of [x, y, w, h] coordinates representing the regions to blur.

    Returns:
        numpy.ndarray: The image with the specified zones blurred.
    """
    blurred_image = image.copy()
    for (x, y, w, h) in zones:
        x = int(x)
        y = int(y)
        w = int(w * 1.2)
        h = int(h * 1.2 )
        # Extract the region of interest (ROI)
        roi = blurred_image[max(0, y-int(h/2)):min(y+int(h/2), image.shape[1])
                          , max(0, x-int(w/2)):min(x+int(w/2), image.shape[0])]
        # Apply Gaussian blur to the ROI
        blurred_roi = cv2.GaussianBlur(roi, (15, 15), 30)
        # Replace the ROI in the image with the blurred ROI
        blurred_image[max(0, y-int(h/2)):min(y+int(h/2), image.shape[1])
                          , max(0, x-int(w/2)):min(x+int(w/2), image.shape[0])] = blurred_roi

    return blurred_image


In [ ]:
# imshow
from google.colab.patches import cv2_imshow

cv2_imshow(blur_zones(image, results[0].boxes.xywh.tolist()))
cv2_imshow(blur_zones(image_2, results_2[0].boxes.xywh.tolist()))

# Make that for whole video

In [ ]:
import cv2
from IPython.display import clear_output
import numpy as np
from time import sleep

# Replace 'video.mp4' with your uploaded video file name
video_path = '/content/doc_2024-12-05_15-06-07.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get video properties for saving the output video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for output video
output_path = '/content/blurred_output_video.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Video finished.")
            break

        # Apply your model to process the frame
        clear_output(wait=True)
        results = model.predict(frame, conf=0.2, iou=1)

        try:
          processed_frame = blur_zones(frame, results[0].boxes.xywh.tolist())
        except:
          processed_frame = frame

        # Write the processed frame to the output video
        out.write(processed_frame)

        # Optionally display the processed frame (comment out if Colab lags)
        # cv2_imshow(processed_frame)

cap.release()
out.release()

print(f"Processed video saved to: {output_path}")



0: 640x640 (no detections), 9.2ms
Speed: 3.2ms preprocess, 9.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
Video finished.
Processed video saved to: /content/blurred_output_video.mp4
